In [44]:
from rsqsim_api.fault.multifault import RsqSimMultiFault
import shutil
from rsqsim_api.catalogue.catalogue import RsqSimCatalogue
import os
import numpy as np
import pandas as pd

seconds_per_year = 31557600.0

In [2]:
# # Tell python where field paths etc are relative to
script_dir = os.path.abspath('')
fault_dir = "../../../data/shaw2021/rundir5091"
catalogue_dir = fault_dir

In [3]:
fault_model = RsqSimMultiFault.read_fault_file_bruce(os.path.join(script_dir, fault_dir, "zfault_Deepen.in"),
                                                     os.path.join(script_dir, fault_dir, "znames_Deepen.in"),
                                                     transform_from_utm=True)
whole_catalogue = RsqSimCatalogue.from_catalogue_file_and_lists(os.path.join(catalogue_dir, "eqs..out"),
                                                                list_file_directory=catalogue_dir, list_file_prefix="catalog")


In [4]:
# identify multifault events
multifault=[ev for ev in whole_catalogue.all_events(fault_model) if ev.num_faults > 1]

In [5]:
#catalogue info
nevents=len(whole_catalogue.event_list)
timeSpan=(np.max(whole_catalogue.patch_time_list) - np.min (whole_catalogue.patch_time_list))/seconds_per_year
nmultifault=len(multifault)
prop_multifault=nmultifault/nevents

In [126]:
#create dictionary of faults in each multifault event

#minimulti=multifault[10:100]
fault_dic={}
i=0
#for ev in minimulti:
for ev in multifault:
    ev_faults=fault_model.filter_faults_by_patch_numbers(ev.patch_numbers)
    ev_fault_names=ev_faults.names
    fault_dic[i]=(ev.event_id,ev,len(ev_fault_names),ev_fault_names)
    i=i+1
multifault_db=pd.DataFrame.from_dict(fault_dic,orient='index',columns=('Event_id','Event','N_faults','Fault_names'))

In [127]:
#sort by number of faults involved
multifault_db.sort_values(by=['N_faults'],axis=0,inplace=True)

In [128]:
# example of search for events with constraints on numbers of faults
two_faults=multifault_db.loc[multifault_db['N_faults']==2]
more_faults=multifault_db.loc[multifault_db['N_faults']>5]
most_faults=multifault_db.loc[multifault_db['N_faults']==np.max(multifault_db['N_faults'])]

In [129]:
#find all multifault events including a specific fault e.g. hundalee (alpiner is a good test because have lots of these)
faultName='hundalee'
#access list of lists of fault names
fault_list=[multifault_db['Fault_names'].values[i] for i in np.arange(0,len(multifault_db['Fault_names'].values))]
#find the instances which contain the fault of interest
fault_events=[fault_list[i] for i in np.where([faultName in fault_list[j] for j in np.arange(0,len(fault_list))])[0]]

In [130]:
len(fault_events)

627